In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("C:\\Users\\Eric Liu\\Desktop\\Purdue\\Sophomore 2nd Semester\\ILS490\\politifact_fake.csv")

In [3]:
tweet_id_list = []
tweet_partition = []
offset = 0
index = int(len(data) / 3)
for i in range(3): 
    tweet_partition.append(data['tweet_ids'].iloc[offset:offset+index-1])
    offset += index

for fold in tweet_partition:
    for group_tweet in data['tweet_ids'].head():
        tweet_list = group_tweet.split("\t")
        for tweet in tweet_list:
            tweet_id_list.append(tweet)
print(len(tweet_id_list))
#tweet_subset = tweet_id_list[400:]
#print(len(tweet_subset))

3171


In [9]:
import time
import requests
import json

# INSERT BEARER TOKEN HERE
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAJUwYwEAAAAAE7TyF7ND4RPHwVD5YEsFtqm5wUI%3D2PAsqcrjWj8QDBg3Gn3NPspRmGLDLplj67kRAhyUvsUdNqNDUm"

#twitter search function
def search_twitter(tweet_id, bearer_token = BEARER_TOKEN):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    url = "https://api.twitter.com/2/tweets/{}?user.fields=public_metrics,created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,url,username,verified,withheld".format(tweet_id)
    response = requests.request("GET", url, headers=headers)

    #print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

all_responses = []
count = 0
tweet_subset = tweet_id_list[200:]
for tweet_id in tweet_subset:
    if count < 200:
        json_response = search_twitter(tweet_id = tweet_id, bearer_token=BEARER_TOKEN)
        print(json_response)
        keys = list(json_response.keys())
        if 'errors' not in keys:
            all_responses.append(json_response)
        count+=1

# Getting dump from the response array
json_dump = json.dumps(all_responses)
# Dumping all the data to a file 
print("appending to file")
outfile = open("C:\\Users\\Eric Liu\\Desktop\\Purdue\\Sophomore 2nd Semester\\ILS490\\fake_data.json", 'a')
outfile.write(json_dump)
outfile.close()
# This would be how you would read the data from the file
#f = open('dumps4/user_data.json')
#data = json.load(f)
#f.close()
 

{'data': {'id': '972954145895297024', 'text': 'FACT CHECK: "Did a Court Order Obama to Pay $400 Million in Restitution?" HELL NO! Another RWNJ Lie! https://t.co/vnz6ILMdbN'}}
{'errors': [{'resource_id': '972960153170468864', 'parameter': 'id', 'resource_type': 'tweet', 'section': 'data', 'title': 'Authorization Error', 'value': '972960153170468864', 'detail': 'Sorry, you are not authorized to see the Tweet with id: [972960153170468864].', 'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource'}]}
{'data': {'id': '972961107664097281', 'text': 'FACT CHECK: Did a Court Order Obama to Pay $400 Million in Restitution?\n\nhttps://t.co/MsvJEsC6Wg\n\nShared from my Google feed'}}
{'data': {'id': '972964621349867520', 'text': 'FACT CHECK: Did a Court Order Obama to Pay $400 Million in Restitution? https://t.co/I9Ir557ZRk'}}
{'data': {'id': '972965859403747329', 'text': 'FACT CHECK: Did a Court Order Obama to Pay $400 Million in Restitution?\n\nhttps://t.co/Pb69MhR1MR\n\nShared 

{'data': {'id': '973176458964283392', 'text': 'FACT CHECK: Did a Court Order Obama to Pay $400 Million in Restitution?\n\nhttps://t.co/pwtbuj0Ms8\n\nShared from my Google feed'}}
{'errors': [{'resource_id': '973177798482321408', 'parameter': 'id', 'resource_type': 'tweet', 'section': 'data', 'title': 'Authorization Error', 'value': '973177798482321408', 'detail': 'Sorry, you are not authorized to see the Tweet with id: [973177798482321408].', 'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource'}]}
{'errors': [{'value': '973189167457980418', 'detail': 'Could not find tweet with id: [973189167457980418].', 'title': 'Not Found Error', 'resource_type': 'tweet', 'parameter': 'id', 'resource_id': '973189167457980418', 'type': 'https://api.twitter.com/2/problems/resource-not-found'}]}
{'data': {'id': '973202052397584386', 'text': 'Who believes this shit in the first place? FACT CHECK: Did a Court Order Obama to Pay $400 Million in Restitution? https://t.co/31vBi8y8t7'}}
{'

Exception: (429, '{"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}')

In [10]:
print(len(all_responses))
print(count)

35
50


In [14]:
data = pd.read_json("C:\\Users\\Eric Liu\\Desktop\\Purdue\\Sophomore 2nd Semester\\ILS490\\user_data.json", orient="records")
info = data.iloc[:,1]

user_list = []
created_at = []
verified = []
protected = []
followers_count = []
following_count = []
tweet_count = []
listed_count = []
for stuff in info:
    if not pd.isna(stuff):
        user_list.append(stuff['username'])
        created_at.append(stuff['created_at'])
        verified.append(stuff['verified'])   
        protected.append(stuff['protected'])
        followers_count.append(stuff['public_metrics']['followers_count'])
        following_count.append(stuff['public_metrics']['following_count'])
        tweet_count.append(stuff['public_metrics']['tweet_count'])
        listed_count.append(stuff['public_metrics']['listed_count'])

new_df = pd.DataFrame(user_list, columns=['User'])
new_df['Created At'] = created_at
new_df['Verified'] = verified
new_df['Protected'] = protected
new_df['Followers Count'] = followers_count
new_df['Following Count'] = following_count
new_df['Tweet Count'] = tweet_count
new_df['Listed Count'] = listed_count
new_df['Misinformation'] = [1] * len(new_df)
new_df['Misinformation'] = 'Fake'
print(len(new_df))

568


In [12]:
check_json = pd.read_json("C:\\Users\\Eric Liu\\Desktop\\Purdue\\Sophomore 2nd Semester\\ILS490\\fake_data.json", orient="records")
print(len(check_json))

108
